In [1]:
import pandas as pd 
import numpy as np
import os 
from isswrapper.util.helpers import read_parquet_into_dataframe
from bs4 import BeautifulSoup
import httpx
from isswrapper.loaders.securities import security_description
from urllib.parse import urljoin
from fuzzywuzzy import fuzz
import bs4
import re

In [2]:
# loading data
current_path = os.getcwd()
project_path = os.path.dirname(current_path)
datasets_folder_path = os.path.join(project_path, 'datasets')
# news 
pnd_token_date_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "pnd_token_date.parquet"))
# all time series
ts_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "time_series_securities_pnd.parquet"))
news_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "final_df.parquet"))
print(ts_df.shape)
ts_df.sample(5)

(135031, 24)


,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR,currencyid
1185,TQBR,2019-02-20,КамчатЭ ап,KCHEP,9.0,27790.0,0.304,0.3,0.326,0.308,...,0.303,0.308,0.0,563620.0,0.0,NaN,3,SUR,5.50,RUB
2162,TQBR,2023-01-04,ТНСэнМарЭл,MISB,85.0,380870.0,15.500,15.4,16.600,15.900,...,15.900,15.900,0.0,509650.0,0.0,0.0,3,SUR,1.92,RUB
1946,TQBR,2022-02-23,ТКЗКК ап,KRKOP,0.0,0.0,NaN,NaN,NaN,9.270,...,9.360,9.270,0.0,500746.0,0.0,0.0,3,SUR,NaN,RUB
960,TQBR,2018-04-04,КрасОкт-ао,KROT,79.0,624640.0,428.000,401.0,428.000,402.000,...,411.000,402.000,624640.0,624640.0,624640.0,NaN,3,SUR,-4.51,RUB
574,TQBR,2016-09-21,ВыбСудЗ ао,VSYD,4.0,19230.0,3210.000,3170.0,3220.000,3170.000,...,3300.000,3170.000,0.0,500860.0,0.0,NaN,3,SUR,-0.94,RUB


In [6]:
def get_sec_names(token: str)->tuple:
    """
    Get short and full names of a security from MOEX security description page.

    :param token: The token name.
    :type token: str
    :return: A tuple containing the short and full names of the security.
    :rtype: tuple
    """
    s_df = security_description(q=token)
    s_df.set_index("name", inplace=True)
    return s_df.loc["SHORTNAME", "value"], s_df.loc["NAME","value"]

def get_smartlab_forum_urls(tokens:list[str], confidence_threshold:int = 90)->list:
    """
    Get forum links from smartlab for given tokens if possible

    :param tokens: list of token to find forum threads
    :type tokens: list[str]
    :param confidence_threshold: similarity threshold used in fuzzywuzzy partial_ratio method, defaults to 90
    :type confidence_threshold: int, optional
    :return: list of tuple filled with token and it full link.
    :rtype: list
    """
    # get names for further search
    sec_names = [get_sec_names(token) for token in tokens]
    # init client connection
    base_url = "https://smart-lab.ru/"
    with httpx.Client(base_url=base_url) as client:
        # soup from sectors page, contains almost all companies
        response = client.get("forum/sectors")
        soup = BeautifulSoup(response.text, "html.parser")
        
        result = []
    
        for token, (s_name, l_name) in zip(tokens, sec_names):
            # extisting endpoints
            response = client.get(f"/forum/{token}")
            if response.status_code == 200:
                result.append((token, f"/forum/{token}"))
                continue
            # slight difference in short names
            tmp = soup.find("a", string=lambda x: x and len(x)>3 and fuzz.partial_ratio(x.lower(), s_name.lower())>=confidence_threshold)
            if tmp:
                result.append((token, tmp.get("href")))
                continue
            # difference in full names 
            tmp = soup.find("a", string=lambda x: x and len(x)>5 and fuzz.partial_ratio(x.lower(), l_name.lower())>=confidence_threshold)
            if tmp:
                result.append((token,  tmp.get("href")))
                continue
            result.append((token,None))
    return result

def generate_forum_thread_page_urls(relative_url: str) -> list[str]:
    """
    Generate URLs for all pages of a certain forum thread.

    :param relative_url: The relative URL of the forum thread, e.g., /forum/ALBK.
    :type relative_url: str
    :return: A list of URLs to parse.
    :rtype: list[str]
    """
    base_url = "https://smart-lab.ru/"
    with httpx.Client(base_url=base_url) as client:
        response = client.get(relative_url)
    soup = BeautifulSoup(response.text, "html.parser")
    last_page = int(soup.find("span", class_="page active").text)
    pages = [urljoin(base_url, relative_url + f"/page{i}") for i in range(1, last_page + 1)]
    return pages

### smartlab stuff


In [6]:
base_url = "https://smart-lab.ru/"

In [5]:
client = httpx.Client(base_url=base_url)

In [ ]:
resp = client.get("forum/sectors")
soup = BeautifulSoup(resp.text, "html.parser")
soup

In [7]:
np.random.randint(0, 40, 5)

array([ 0, 31, 13, 12, 18])

In [8]:
tokens = pnd_token_date_df["token"].unique().tolist()
result = get_smartlab_forum_urls([tokens[i] for i in np.random.randint(0, len(tokens), 5)])
result

[('RLMNP', '/forum/RLMN'),
 ('BLNG', '/forum/BLNG'),
 ('PAZA', '/forum/PAZA'),
 ('ABRD', '/forum/ABRD'),
 ('VGSBP', '/forum/VGSB')]

next step is to get last page number and generate url for all pages. Pages on smartlab works as /forum/{security name}/page{numpage}
but first step is to extract number of last page. On default if you go to specified forum page you ll be on the lastmost page. So we need to go through all links and take page numbers, then create for every thread list filled with pages

In [10]:
client = httpx.Client(base_url=base_url)

In [17]:
pre_f_df = []
for token, url in result:
    page_list = generate_forum_thread_page_urls(url)
    url_df = pd.DataFrame({"token":[token]*len(page_list), "url":page_list})
    pre_f_df.append(url_df)
f_df = pd.concat(pre_f_df)
f_df.sample(5)

,token,url
18,ABRD,https://smart-lab.ru/forum/ABRD/page19
0,VGSBP,https://smart-lab.ru/forum/VGSB/page1
0,RLMNP,https://smart-lab.ru/forum/RLMN/page1
24,ABRD,https://smart-lab.ru/forum/ABRD/page25
3,RLMNP,https://smart-lab.ru/forum/RLMN/page4


In [21]:
client = httpx.Client()
resp = client.get("https://smart-lab.ru/forum/ABRD/page19")


In [23]:
str(resp.url)

'https://smart-lab.ru/forum/ABRD/page19'

In [19]:
f_df["url"]

0    https://smart-lab.ru/forum/RLMN/page1
1    https://smart-lab.ru/forum/RLMN/page2
2    https://smart-lab.ru/forum/RLMN/page3
3    https://smart-lab.ru/forum/RLMN/page4
4    https://smart-lab.ru/forum/RLMN/page5
                     ...                  
3    https://smart-lab.ru/forum/VGSB/page4
4    https://smart-lab.ru/forum/VGSB/page5
5    https://smart-lab.ru/forum/VGSB/page6
6    https://smart-lab.ru/forum/VGSB/page7
7    https://smart-lab.ru/forum/VGSB/page8
Name: url, Length: 67, dtype: object

In [ ]:
soup = BeautifulSoup(resp.text,"html.parser")
soup

In [110]:
last_page = int(soup.find("span", class_="page active").text)
pages = [urljoin(base_url, example+f"/page{i}") for i in range(1, last_page+1)]
pages[:5]

['https://smart-lab.ru/forum/BELU/page1',
 'https://smart-lab.ru/forum/BELU/page2',
 'https://smart-lab.ru/forum/BELU/page3',
 'https://smart-lab.ru/forum/BELU/page4',
 'https://smart-lab.ru/forum/BELU/page5']

In [109]:
resp2 = client.get(pages[0])
resp2.status_code

200

In [113]:
result[-5][1]

'/forum/VRSB'

In [116]:
to_parse = generate_forum_thread_page_urls(result[-15][1])

In [128]:
url = to_parse[5]

url

'https://smart-lab.ru/forum/TGKB/page6'

In [122]:
client = httpx.Client()

In [123]:
response=client.get(url)
response.status_code

In [125]:
soup = BeautifulSoup(response.text, "html.parser")

In [126]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<title>Форум акции ТГК-2 (TGKB, TGKBP), страница 6</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Обсуждение и комментарии инвесторов по акциям ТГК-2. Прогноз курса акций ТГК-2 страница 6" name="DESCRIPTION"> <meta content="ТГК-2, обсуждение компании, информация о компании, новости компании." name="KEYWORDS"/> <meta content="69df339e9279f161" name="yandex-verification"/>
<meta content="17fde70f-5d0a-4de9-809e-a9f5334ce8dd" name="PartnerFinam"/>
<link href="https://mc.yandex.ru" rel="preconnect"/>
<link href="//counter.yadro.ru" rel="preconnect"/>
<link href="https://www.googletagmanager.com" rel="preconnect"/>
<link href="https://yastatic.net" rel="dns-prefetch"/>
<link href="https://content.mql5.com" rel="dns-prefetch"/>
<meta content="no" name="push-subscribes"/>
<meta content="353607944842117" property="fb:app_id"><meta content="

In [131]:
comments = soup.find_all("li", attrs={"data-type":"comment"})

In [172]:
type(comment)

bs4.element.Tag

In [48]:
[np.random.choice(tokens) for i in range(5)]


['LNZLP', 'IDVP', 'BELU', 'TGKN', 'MISB']

In [180]:
comment = comments[0]
comment

<li class="cm_wrap bluid_31628" data-id="10645580" data-type="comment"><a name="comment10645580"></a><a aria-label="Профиль Victor Glukhov" href="/profile/Rbaker/" rel="nofollow"><img alt="Аватар Victor Glukhov" class="avtr_box" loading="lazy" src="/uploads/2021/images/03/16/28/2021/10/11/avatar_100x100.webp?4398"/></a><div class="cmt_body"><div><span><a class="a_name trader_other" href="/profile/Rbaker/" rel="nofollow">Victor Glukhov</a></span><a class="a_time" href="/forum/TGKB/goto_comment_10645580/#comment10645580" rel="nofollow" target="_blank"><time datetime="2020-02-13T11:21:11+03:00">13 февраля 2020, 11:21</time></a></div><div class="text">С июля 2018-ого, до февраля 2020-ого цена почти не менялась.<br/>
Если смотреть на графике (месяц) — всё это выглядит как прямая нитка. Вот и долгожданное сильное движение цены.<br/>
Такая же история была перед ростов в 2017-ом.</div><div class="cm_ftr"><a class="reply" href="/login/" rel="nofollow">Ответить</a><a class="cm_ico th up" href="/

In [138]:
cmt_body = comment.find("div", class_="cmt_body")
cmt_body

<div class="cmt_body"><div><span><a class="a_name trader_other" href="/profile/Malikeldjebena/" rel="nofollow">Malik</a></span><a class="a_time" href="/forum/TGKB/goto_comment_9738142/#comment9738142" rel="nofollow" target="_blank"><time datetime="2019-05-20T16:53:54+03:00">20 мая 2019, 16:53</time></a></div><div class="text"><blockquote class="reply"><blockquote class="reply"><br/>
«ТГК-2» — около 4 млрд рублей.<br/>
</blockquote><br/>
редактор Боб, наверное, все-таки огк-2</blockquote><br/>
bayad, для аналитегов политизированной помойки финанза это одно и то же )</div><div class="cm_ftr"><a class="reply" href="/login/" rel="nofollow">Ответить</a><a class="cm_ico th up" href="/login/" rel="nofollow"></a><a class="cm_ico th dn" href="/login/" rel="nofollow"></a><a class="cm_mrk" href="/login/" rel="nofollow"> 0</a></div></div>

In [140]:
cmt_text = cmt_body.find("div", class_="text")
likes = cmt_body.find("div", class_="cm_ftr")
likes

<div class="cm_ftr"><a class="reply" href="/login/" rel="nofollow">Ответить</a><a class="cm_ico th up" href="/login/" rel="nofollow"></a><a class="cm_ico th dn" href="/login/" rel="nofollow"></a><a class="cm_mrk" href="/login/" rel="nofollow"> 0</a></div>

In [142]:
cmt_text.text

'\r\n«ТГК-2» — около 4 млрд рублей.\n\r\nредактор Боб, наверное, все-таки огк-2\r\nbayad, для аналитегов политизированной помойки финанза это одно и то же )'

In [149]:
int(likes.find("a", class_="cm_mrk").text)

0

In [175]:
cmt_body.find("a", class_="cm_mrk").text

' 0'

In [33]:
import datetime
import arrow
import bs4

In [160]:
cmt_time = arrow.get(cmt_body.find("time").get("datetime")).datetime
cmt_time

datetime.datetime(2019, 5, 20, 16, 53, 54, tzinfo=tzoffset(None, 10800))

In [167]:
user_id = cmt_body.find("a", class_="a_name trader_other").get("href").split("/")[2]
user_id

'Malikeldjebena'

In [171]:
entitiled = not cmt_body.find("a", class_="image_true") is None
entitiled

False

In [41]:
def extract_comment_data(comment: bs4.element.Tag) -> dict:
    """
    Extract valuable data from a comment and package it into a dictionary.

    :param comment: The comment element.
    :type comment: bs4.element.Tag
    :return: Extracted data as a dictionary.
    :rtype: dict
    """
    comment_body = comment.find("div", class_="cmt_body")
    return dict(
        comment_text = comment_body.find("div", class_="text").text,
        comment_score = int(comment_body.find("a", class_="cm_mrk").text.replace(" ", "")),
        comment_datetime = arrow.get(comment_body.find("time").get("datetime")).datetime,
        user_id = comment_body.find("a", class_="a_name trader_other").get("href").split("/")[2],
        badges = not comment_body.find("a", class_="image_true") is None,
        )

In [27]:

resp.text

'<!DOCTYPE html>\n<html lang="ru">\n\t<head>\n\t\t<title>Форум акции Абрау-Дюрсо (ABRD), страница 19</title>\n\t\t<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1">\n\t\t<meta name="DESCRIPTION" content="Обсуждение и комментарии инвесторов по акциям Абрау-Дюрсо. Прогноз курса акций Абрау-Дюрсо страница 19"/>\t\t<meta name="KEYWORDS" content="Абрау-Дюрсо, обсуждение компании, информация о компании, новости компании."/>\t\t\t\t<meta name=\'yandex-verification\' content=\'69df339e9279f161\' />\n\t\t<meta name="PartnerFinam" content="17fde70f-5d0a-4de9-809e-a9f5334ce8dd">\n\t\t\n\t\t<link rel="preconnect" href="https://mc.yandex.ru">\n\t\t<link rel="preconnect" href="//counter.yadro.ru">\n\t\t<link rel="preconnect" href="https://www.googletagmanager.com">\n\t\t\t\t<link rel="dns-prefetch" href="https://yastatic.net">\n\t\t\t\t\t\t<link rel="dns-prefetch" href="https://content.mql5.com">\n\t\t\n\t\t

In [28]:
f_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "test.parquet"))
f_df.head(5)

,token,url,body
0,ABRD,https://smart-lab.ru/forum/ABRD/page1,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
1,ABRD,https://smart-lab.ru/forum/ABRD/page2,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
2,ABRD,https://smart-lab.ru/forum/ABRD/page3,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
3,ABRD,https://smart-lab.ru/forum/ABRD/page4,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."
4,ABRD,https://smart-lab.ru/forum/ABRD/page5,"<!DOCTYPE html>\n<html lang=""ru"">\n\t<head>\n\..."


In [29]:
bodys = f_df["body"]
len(bodys)

33

In [46]:
f_df["token"].unique().tolist()

['ABRD', 'PAZA']

In [42]:
pre_fin_list = []
for idx, row in f_df.iterrows():
    soup = BeautifulSoup(row["body"])
    comments = soup.find_all("li", attrs={"data-type":"comment"})
    f_list = []
    for comment in comments:
        f_list.append(extract_comment_data(comment))
    n_df = pd.DataFrame(f_list)
    n_df["token"] = row["token"]
    n_df["url"] = row["url"]
    pre_fin_list.append(n_df)
ffff_df = pd.concat(pre_fin_list)
    

,comment_text,comment_score,comment_datetime,user_id,badges,token,url
0,Павловский автобус - Цена выкупа 6339 руб за о...,0,2021-10-28 09:43:03+03:00,Marek,False,PAZA,https://smart-lab.ru/forum/PAZA/page5
15,"Распределить прибыль ПАО «Абрау – Дюрсо», полу...",0,2021-05-24 16:22:53+03:00,Marek,False,ABRD,https://smart-lab.ru/forum/ABRD/page10
3,"Иван Соловьев, есть доказательства, что это ма...",0,2023-07-10 10:32:00+03:00,IvanSolovev,False,PAZA,https://smart-lab.ru/forum/PAZA/page7
23,Этого напитка не хватает в России… Водяра косо...,0,2023-01-17 15:10:09+03:00,Ds10,False,ABRD,https://smart-lab.ru/forum/ABRD/page16
24,Что за чудеса с этой компанией? Новый биткоин?...,1,2019-09-26 17:59:31+03:00,Andy24pro,False,PAZA,https://smart-lab.ru/forum/PAZA/page2
4,Популярность зарубежного вина достигла рекордн...,0,2023-10-03 07:41:38+03:00,Buterbrod2018,False,ABRD,https://smart-lab.ru/forum/ABRD/page25
8,Имущество «ПАЗ»а давно заложено «ГАЗ»ом в обес...,0,2020-06-14 20:25:58+03:00,WBaffet,False,PAZA,https://smart-lab.ru/forum/PAZA/page4
5,"Когда МСФО 2019 год?\r\nkhornickjaadle, удовле...",0,2020-04-08 06:38:42+03:00,khornickjaadle,False,ABRD,https://smart-lab.ru/forum/ABRD/page4
24,Формулировка решения по первому вопросу повест...,0,2023-01-25 22:55:15+03:00,Marek,False,PAZA,https://smart-lab.ru/forum/PAZA/page7
1,Походу аграрное лобби сдувается… \r\nМой прогн...,0,2020-06-15 19:21:25+03:00,FinOrg,False,ABRD,https://smart-lab.ru/forum/ABRD/page4


In [43]:
ffff_df.to_parquet("penis.parquet")

In [39]:
n_df["token"] = 
n_df.head()

,comment_text,comment_score,comment_datetime,user_id,badges
0,"Auximen, а её вообще дают шортить?",0,2017-07-02 13:49:10+03:00,romanranniy,False
1,"Я бы зашортил с текущей цены до 150, но ABRD «...",0,2017-07-02 13:47:41+03:00,Auximen,False
2,"сегодня ожидаем: ABRD: отсечка по д-дам 1,29 р...",0,2017-07-02 00:30:00+03:00,dhc,False
3,"сегодня ожидаем: ABRD: посл. день с дивид. 1,2...",0,2017-06-28 00:30:01+03:00,dhc,False
4,"Ну вот, а сегодня в обратную сторону.",0,2017-06-22 17:54:49+03:00,dimalar,False


In [183]:
decompose_comment(comment)

{'comment_text': 'С июля 2018-ого, до февраля 2020-ого цена почти не менялась.\r\nЕсли смотреть на графике (месяц) — всё это выглядит как прямая нитка. Вот и долгожданное сильное движение цены.\r\nТакая же история была перед ростов в 2017-ом.',
 'comment_score': 0,
 'comment_datetime': datetime.datetime(2020, 2, 13, 11, 21, 11, tzinfo=tzoffset(None, 10800)),
 'user_id': 'Malikeldjebena',
 'badges': False}

### mfd stuff

In [106]:
base_url="https://lite.mfd.ru"

In [3]:
tokens = pnd_token_date_df.sample(7)["token"].tolist()
tokens

['DZRD', 'GTLC', 'VGSBP', 'VRSB', 'LNZL', 'ROST', 'IGSTP']

In [4]:
tokens = pnd_token_date_df["token"]

In [7]:
t_names = [get_sec_names(token) for token in tokens]
t_names
# retry when moex is up

[('АбрауДюрсо', 'Абрау-Дюрсо ПАО ао'),
 ('БестЭфБ ао', 'Бест Эффортс Банк ПАО ао'),
 ('АстрЭнСб', '"Астраханская ЭСК" ПАО'),
 ('НоваБев ао', 'НоваБев Групп ПАО ао'),
 ('Белон ао', 'Белон ОАО ао'),
 ('БСП ап', 'Банк Санкт-Петербург ап'),
 ('ЧКПЗ ао', '"ЧКПЗ" ПАО ао'),
 ('ДагСб ао', 'ао Дагестанская энергосб.комп.'),
 ('ДЭК ао', '"ДЭК" ПАО ао'),
 ('ДонскЗР', 'Донской завод радиодеталей ао'),
 ('Электрцинк', 'Электроцинк ПАО ао'),
 ('FXRW ETF', 'FinEx RUB GLOBAL EQUITY UC ETF'),
 ('FXWO ETF', 'FinEx USD GLOBAL EQUITY UC ETF'),
 ('iГЕНЕТИКО', 'ЦГРМ ГЕНЕТИКО ао'),
 ('ПАОДжиТиЭл', 'ПАО "ДжиТиЭл" ао'),
 ('ГТМ ао', 'ПАО "ГТМ" ао'),
 ('Инв-Девел', 'ПАО "ИНВЕСТ-ДЕВЕЛОПМЕНТ" - ао'),
 ('Ижсталь2ао', 'Ижсталь ПАО ао 2в.'),
 ('Ижсталь ап', 'Ижсталь ПАО ап'),
 ('iАРТГЕН ао', 'ПАО "Артген"'),
 ('КамчатЭ ао', 'Камчатскэнерго ПАО ао'),
 ('КамчатЭ ап', 'Камчатскэнерго ПАО ап'),
 ('ТКЗКК ап', 'ТКЗ Красный котельщик ПАО ап'),
 ('КрасОкт-ао', '"КрасныйОктябрь" ПАО - ао'),
 ('КрасОкт-1п', 'КрасныйОктябрь-1п'

In [8]:
df = pd.DataFrame(t_names)

In [10]:
df.columns = ["shortname", "longname"]

In [12]:
df["token"] = tokens

In [14]:
df.to_parquet(os.path.join(datasets_folder_path, "sec_names.parquet"))

In [108]:
token = np.random.choice(tokens)

In [5]:
def get_mfd_thread_links(tokens):
    # get most visited thread for given token(s)
    result = dict()
    with httpx.Client(base_url="https://lite.mfd.ru") as client:
        for token in tokens:
            response = client.get("/forum/subforum/", params=dict(id=1, q=token))
            soup = BeautifulSoup(response.text, "html.parser")
            search_result_table = soup.find("table", class_="mfd-table mfd-threads")
            wrapped_link = search_result_table.find("td", class_="mfd-item-subject")
            link=None
            if wrapped_link:
                link = wrapped_link.find("a").get("href")
            # wrapped_links = search_result_table.find("td", class_="mfd-item-subject")
            # links = [link.find("a").get("href") for link in wrapped_links]
            result[token]=link
    return result
# shortname and long name usage required
# a lot to go, a lot to do

In [109]:
link_lists = get_mfd_thread_links(tokens)
link_lists

{'KCHE': '/forum/thread/?id=63489&2022',
 'SVET': '/forum/thread/?id=111519&2022',
 'DVEC': '/forum/thread/?id=62197&2022',
 'RTSB': None,
 'RKKE': '/forum/thread/?id=61551&2022',
 'SVAV': '/forum/thread/?id=61394&2022',
 'KCHEP': None}

In [7]:
from pnd_moex.general.scraper import generate_forum_thread_page_urls, extract_comments_from_page

In [110]:
test_urls = generate_forum_thread_page_urls(
    base_url,
    list(link_lists.values())[0],
    last_page=dict(name="a", class_="mfd-paginator-selected"),
    page_fstr="&page={i}",
    zero_index=True
    )
test_urls

['https://lite.mfd.ru/forum/thread/?id=63489&2022&page=0',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=1',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=2',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=3',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=4',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=5',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=6',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=7',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=8',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=9',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=10',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=11',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=12',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=13',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=14',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=15',
 'https://lite.mfd.ru/forum/thread/?id=63489&2022&

In [132]:
def extract_mfd_comment_data(comment: bs4.element.Tag) -> dict:
    """
    Extract valuable data from a comment and package it into a dictionary.
    :param comment: The comment element.
    :type comment: bs4.element.Tag
    :return: Extracted data as a dictionary.
    :rtype: dict
    """
    comment_misc = comment.find("div", class_="mfd-post-remark")
    user_id = comment.find("a", class_="mfd-poster-link")
    comment_score = comment.find("span", class_="u")
    user_score = comment.find("div", class_="mfd-poster-info-rating mfd-icon-profile-star")
    comment_text = comment.find("div", class_="mfd-post-text")
    return dict(
        comment_text=None if comment_text is None else comment_text.text,
        comment_score=0 if comment_score is None else int(comment_score.text),
        comment_datetime=arrow.get(comment.find("a", class_="mfd-post-link").text, 'DD.MM.YYYY HH:mm').datetime,
        comment_misc = None if comment_misc is None else comment_misc.text,
        user_id = None if user_id is None else user_id.get("href"),
        user_score = None if user_score is None else int(re.search(r'\((\d+)\)',user_score.find("a").get("title")).group(1)),
    )


In [111]:
proba_url = test_urls[0]
proba_url

'https://lite.mfd.ru/forum/thread/?id=63489&2022&page=0'

In [120]:
with httpx.Client() as client:
    resp = client.get(proba_url)

In [133]:
af_df = extract_comments_from_page(resp, dict(name="div", class_="mfd-post"), extract_mfd_comment_data)
af_df.sample(5)

,comment_text,comment_score,comment_datetime,comment_misc,user_id,user_score
0,"Братья и сестры рад приветствовать вас, вот ре...",0,2010-11-26 12:56:00+00:00,Сообщение перенесено администратором МФД 29.11...,None,NaN
12,"Вам на рынке важней брать как вам кажется ""хор...",0,2010-11-28 18:38:00+00:00,None,/forum/poster/?id=59135,2522.0
15,Моя родина - свобода. @ 28.11.2010 18:44Спаси...,0,2010-11-28 18:49:00+00:00,None,/forum/poster/?id=58862,43504.0
14,Спасибо! не пробьет выйдем! Удачи!,0,2010-11-28 18:44:00+00:00,None,/forum/poster/?id=59135,2522.0
2,СПРАВКА: ОАО «Камчатскэнерго» создано в 1964...,0,2010-11-28 17:15:00+00:00,None,/forum/poster/?id=59135,2522.0


In [124]:
soup = BeautifulSoup(resp.text, "html.parser")
tmp = soup.find_all(**dict(name="div", class_="mfd-post"))[4]


In [118]:
extract_mfd_comment_data(tmp)


{'comment_text': 'И последнее добавляю с сайта решение собрания акционеров:  ПРОЕКТЫ РЕШЕНИЙ ВНЕОЧЕРЕДНОГО ОБЩЕГО СОБРАНИЯ АКЦИОНЕРОВ   ОАО «КАМЧАТСКЭНЕРГО»   Вопрос №1: О реорганизации Открытого акционерного общества энергетики и электрификации «Камчатскэнерго» в форме присоединения к нему Открытого акционерного общества «Камчатские ТЭЦ», Открытого акционерного общества «Центральные электрические сети», Открытого акционерного общества «Камчатская тепло-транспортная компания»    Проект решения:  1. Реорганизовать Открытое акционерное общество энергетики и электрификации «Камчатскэнерго» в форме присоединения к нему Открытого акционерного общества «Камчатские ТЭЦ», Открытого акционерного общества «Центральные электрические сети», Открытого акционерного общества «Камчатская тепло-транспортная компания» на условиях, предусмотренных Договором о присоединении Открытого акционерного общества «Камчатские ТЭЦ», Открытого акционерного общества «Центральные электрические сети», Открытого акционе

In [98]:
tmp.find("table")


<table><tr><td><a href="/forum/poster/?id=65375" rel="nofollow">Оптовик</a><td>+</td></td></tr></table>

In [127]:
import re

In [130]:
int(match.group(1))

'2522'

In [86]:
tmp.find("span", class_="u").text

AttributeError: 'NoneType' object has no attribute 'text'

In [79]:
tmp.find("div",class_="mfd-post-top-2").text


'1Оптовик+'

In [70]:
int(tmp.find("div", class_="mfd-post-body-left").text)

ValueError: invalid literal for int() with base 10: ''

In [60]:
tmp.find("div", class_="mfd-post-text").text

'на чем растет???'

In [61]:
tmp.find("div", class_="mfd-post-remark").text

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
extract_comments_from_page(resp, comment_dict=dict(name="div", class_="mfd-post"), )

In [1]:
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
t = np.linspace()

### e-disclosure stuff


In [18]:
tokens = pnd_token_date_df.sample(5)["token"].tolist()
tokens

['PAZA', 'IGSTP', 'ISKJ', 'LVHK', 'BLNG']

In [3]:
sn_df = pd.read_parquet(os.path.join(datasets_folder_path, "sec_names.parquet"))
sn_df.sample(5)

,shortname,longname,token
5,БСП ап,Банк Санкт-Петербург ап,BSPBP
7,ДагСб ао,ао Дагестанская энергосб.комп.,DASB
3,НоваБев ао,НоваБев Групп ПАО ао,BELU
24,КрасОкт-1п,КрасныйОктябрь-1п,KROTP
20,КамчатЭ ао,Камчатскэнерго ПАО ао,KCHE


In [24]:
tokens = sn_df["token"].unique().tolist()

In [23]:
from pnd_moex.general.scraper import get_smartlab_forum_urls
import httpx


In [26]:
raw_url_df_list = []
base_url = "https://smart-lab.ru/"
# Token forum pages
forum_token_endpoints_1 = get_smartlab_forum_urls(tokens, 95)
forum_token_endpoints_1[:5]

AttributeError: 'list' object has no attribute 'head'

In [27]:
forum_token_endpoints_1


[('ABRD', '/forum/ABRD'),
 ('ALBK', '/forum/ALBK'),
 ('ASSB', '/forum/ASSB'),
 ('BELU', '/forum/BELU'),
 ('BLNG', '/forum/BLNG'),
 ('BSPBP', '/forum/BSPB'),
 ('CHKZ', '/forum/CHKZ'),
 ('DASB', '/forum/DASB'),
 ('DVEC', '/forum/DVEC'),
 ('DZRD', '/forum/DZRD'),
 ('ELTZ', '/forum/ELTZ'),
 ('FXRW', '/forum/FXRW'),
 ('FXWO', '/forum/FXWO'),
 ('GECO', '/forum/GECO'),
 ('GTLC', '/forum/GTLC'),
 ('GTRK', '/forum/GTRK'),
 ('IDVP', '/forum/IDVP'),
 ('IGST', '/forum/IGST'),
 ('IGSTP', '/forum/IGST'),
 ('ISKJ', '/forum/ISKJ'),
 ('KCHE', '/forum/KCHE'),
 ('KCHEP', '/forum/KCHE'),
 ('KRKOP', '/forum/KRKO'),
 ('KROT', '/forum/KROT'),
 ('KROTP', None),
 ('KTSB', '/forum/KTSB'),
 ('KTSBP', None),
 ('KUBE', '/forum/KUBE'),
 ('KUZB', '/forum/KUZB'),
 ('LENT', '/forum/LENT'),
 ('LNZL', '/forum/LNZL'),
 ('LNZLP', '/forum/LNZL'),
 ('LPSB', '/forum/LPSB'),
 ('LVHK', '/forum/LVHK'),
 ('MERF', '/forum/MERF'),
 ('MGVM', '/forum/MGVM'),
 ('MISB', '/forum/MISB'),
 ('MISBP', '/forum/MISB'),
 ('MOBB', '/forum/MOBB

In [ ]:
# вписать сюда дополнение к названиям с формумов.
# пофиксить залупу с хуёвыми ссылками
# ну и подумать как всё собрать вместе и сохранить в 

In [ ]:
sn_df.head(5)

In [5]:
client = httpx.Client()

In [9]:
url = 'https://www.e-disclosure.ru/api/search/companies'
data = {
    'textfield': 'КрасОкт-1п',
    'radReg': 'FederalDistricts',
    'districtsCheckboxGroup': '-1',
    'regionsCheckboxGroup': '-1',
    'branchesCheckboxGroup': '-1',
    'lastPageSize': '10',
    'lastPageNumber': '1',
    'query': 'КрасОкт-1п',
}

# Выполните POST-запрос
response = client.post(url, data=data, follow_redirects=True)
response.status_code

200

In [10]:
response.content

b'{"foundCompaniesList":[],"pagingInfo":{"totalItems":0,"itemsPerPage":10,"currentPage":1,"totalPages":0},"allFoundCompanies":0,"maxFoundCompaniesToShow":400}'

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [12]:
driver = webdriver.Firefox()

In [13]:
driver.implicitly_wait(10)

In [14]:
driver.get("https://www.e-disclosure.ru/poisk-po-kompaniyam")

In [28]:
elem = driver.find_element(By.TAG_NAME, "input")
elem

<selenium.webdriver.remote.webelement.WebElement (session="b4daa431-7dae-4e82-bc1b-2c4558bce2ed", element="e10521af-821c-4033-9c19-2d7d82cb9cef")>

In [29]:
elem.send_keys("Мультисис")


ElementNotInteractableException: Message: Element <input name="__RequestVerificationToken" type="hidden"> is not reachable by keyboard
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:349:5
webdriverSendKeysToElement@chrome://remote/content/marionette/interaction.sys.mjs:629:13
interaction.sendKeysToElement@chrome://remote/content/marionette/interaction.sys.mjs:603:11
sendKeysToElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:533:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:150:31
